In [8]:
import json
import pandas as pd
import gzip
import os

# ==================== ĐỌC FILE JSONL ====================
file_path = r"C:\Users\vinh\Desktop\bs4\data\traffic_2025_clean.jsonl"

data = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            obj = json.loads(line)
            data.append(obj)
        except Exception as e:
            print("Lỗi đọc dòng:", e)

df = pd.DataFrame(data)

print("Tổng số bài:", len(df))
print("Các cột có trong file:")
print(df.columns.tolist())
print("Xem thử vài dòng đầu:")
print(df.head(2))

# ==================== ĐỌC NỘI DUNG HTML ====================
def read_html_auto(path):
    # kiểm tra file có tồn tại hay không
    if not os.path.exists(path):
        print("⚠️ Không tìm thấy file:", path)
        return ""
    
    try:
        # đọc 2 byte đầu để nhận dạng có bị nén gzip không
        with open(path, "rb") as f:
            head = f.read(2)

        # nếu có nén gzip
        if head == b"\x1f\x8b":
            with gzip.open(path, "rt", encoding="utf-8", errors="ignore") as fin:
                html_code = fin.read()
        else:
            # nếu không nén
            with open(path, "rt", encoding="utf-8", errors="ignore") as fin:
                html_code = fin.read()
        return html_code
    except Exception as e:
        print("❌ Lỗi đọc file HTML:", e)
        return ""

# tạo cột mới để lưu nội dung HTML
df["html_content"] = df["raw_html_path"].apply(read_html_auto)

print("Ví dụ nội dung HTML gốc (1000 ký tự đầu):")
print(df["html_content"].iloc[0][:1000])

# ==================== KIỂM TRA ====================
print("Đã đọc xong toàn bộ file HTML, có thể xử lý nội dung tiếp theo.")


Tổng số bài: 2746
Các cột có trong file:
['url', 'title', 'pub_date', 'year', 'preview', 'raw_html_path']
Xem thử vài dòng đầu:
                                                 url  \
0  https://nld.com.vn/tien-ve-clb-tp-hcm-ghi-ban-...   
1  https://nld.com.vn/than-y-cua-tuyen-viet-nam-t...   

                                               title    pub_date    year  \
0  Tiền vệ CLB TP HCM ghi bàn thắng đẹp nhất thán...  2025-02-12  2025.0   
1     "Thần y" của tuyển Việt Nam tại ASEAN Cup 2024  2025-01-27  2025.0   

                                             preview  \
0  VPF vừa công bố các danh hiệu giành cho đội bó...   
1  Trong bóng đá, bên cạnh những màn trình diễn d...   

                                       raw_html_path  
0  raw_html\20251010\2c3c1d99e184152436010a92258d...  
1  raw_html\20251010\668441722221dc12012bc72f3e07...  
Ví dụ nội dung HTML gốc (1000 ký tự đầu):
<!DOCTYPE html>
<html lang="vi">
<head>
    <meta http-equiv="Content-Type" content="text/html; ch

In [12]:
# ==================== LƯU FILE ====================
out_file = r"C:\Users\vinh\Desktop\bs4\data\traffic_html_full.jsonl"

with open(out_file, "w", encoding="utf-8") as f:
    for i, row in df.iterrows():
        rec = {
            
            "html_content": row["html_content"]
        }
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("✅ Đã lưu toàn bộ nội dung HTML vào:", out_file)


KeyboardInterrupt: 

In [15]:
import json
from bs4 import BeautifulSoup

input_file = "traffic_html_full.jsonl"
output_file = "tienxuly_cleaned.jsonl"

with open(input_file, "r", encoding="utf-8") as fin, \
     open(output_file, "w", encoding="utf-8") as fout:

    for line in fin:
        try:
            obj = json.loads(line)
            html = obj.get("html_content", "")
            soup = BeautifulSoup(html, "html.parser")

            # --- Tiêu đề ---
            title_tag = soup.find("h1", class_="title")
            title = title_tag.text.strip() if title_tag else ""

            # --- Mô tả ngắn ---
            summary_tag = soup.find("p", class_="sapo")
            summary = summary_tag.text.strip() if summary_tag else ""

            # --- Nội dung bài viết ---
            body_tag = soup.find("div", class_="contentdetail")
            if body_tag:
                paragraphs = [p.get_text(" ", strip=True) for p in body_tag.find_all("p") if p.get_text(strip=True)]
                content = "\n".join(paragraphs)
            else:
                content = ""

            # --- Gộp kết quả ---
            result = {
                "title": title,
                "summary": summary,
                "content": content
            }

            fout.write(json.dumps(result, ensure_ascii=False) + "\n")

        except Exception as e:
            print("Lỗi:", e)

print("✅ Đã xử lý xong! File kết quả:", output_file)


✅ Đã xử lý xong! File kết quả: tienxuly_cleaned.jsonl


In [11]:
import json
from bs4 import BeautifulSoup

# ==================== ĐỌC FILE JSONL ====================
file_path = r"C:\Users\vinh\Desktop\bs4\data\traffic_html_full.jsonl"

data = []
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            obj = json.loads(line)
            data.append(obj)
        except Exception as e:
            print("Lỗi đọc dòng:", e)

print("Tổng số bài:", len(data))

# ==================== LÀM SẠCH NỘI DUNG HTML ====================
cleaned_news = []

for item in data:
    try:
        html = item.get("html_content", "")

        soup = BeautifulSoup(html, "html.parser") # or 'lxml'

        # --- Crawl dữ liệu ---
        # Tiêu đề
        title_tag = soup.find('h1', class_='title-detail')
        title = title_tag.text.strip() if title_tag else None

        # Thời gian
        time_tag = soup.find('div', class_='header-content width_common')
        time_info = time_tag.text.strip() if time_tag else None

        # Tóm tắt
        summary_tag = soup.find('p', class_='description')
        summary = summary_tag.text.strip() if summary_tag else None

        # Nội dung bài viết
        content_tag = soup.find('article', class_='fck_detail ')
        if content_tag:
            paragraphs = [p.text.strip() for p in content_tag.find_all('p') if p.text.strip()]
            content = "\n".join(paragraphs)
        else:
            content = None

        # Lấy cả phần nội dung mở rộng
        full_noi_dung_tag = soup.find('div', class_='sidebar-1')
        full_noi_dung = full_noi_dung_tag.text.strip() if full_noi_dung_tag else None

        news_data = {
            "url": url,
            "title": title,
            "time": time_info,
            "summary": summary,
            "content": full_noi_dung if full_noi_dung else content
        }

        cleaned_news.append(news_data)

    except Exception as e:
        print("Lỗi xử lý bài:", e)

# ==================== LƯU KẾT QUẢ ====================
with open(r"C:\Users\vinh\Desktop\bs4\news_cleaned.jsonl", "w", encoding="utf-8") as f:
    for item in cleaned_news:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print("✅ Đã xử lý xong và lưu vào file: news_cleaned.jsonl")


Tổng số bài: 345
✅ Đã xử lý xong và lưu vào file: news_cleaned.jsonl
